In [1]:
import keras
import numpy as np
import tensorflow as tf
from sklearn.metrics import accuracy_score, confusion_matrix, precision_recall_fscore_support, roc_auc_score

class_names = ["confirmed", "crossedout", "empty"]

2024-09-11 20:52:22.942410: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-11 20:52:22.959117: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-11 20:52:22.964852: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-09-11 20:52:22.978981: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-09-11 20:52:23.853265: W tensorflow/compiler/tf2

In [2]:
model = keras.saving.load_model("models/omr_model.keras")
test_ds =  tf.data.Dataset.load("datasets/ds_test")

I0000 00:00:1726098745.408873    4976 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1726098745.433038    4976 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1726098745.433099    4976 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1726098745.435091    4976 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1726098745.435227    4976 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:0

In [3]:
true_labels = []
for _, label in test_ds:
    true_labels.append(label.numpy())

true_labels = np.concatenate(true_labels)

2024-09-11 20:52:29.698140: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [4]:
results = model.predict(test_ds)

I0000 00:00:1726098750.613953    5055 service.cc:146] XLA service 0x7ff740002d40 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1726098750.614029    5055 service.cc:154]   StreamExecutor device (0): NVIDIA GeForce GTX 1660, Compute Capability 7.5
2024-09-11 20:52:30.649485: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-09-11 20:52:32.496635: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:531] Loaded cuDNN version 8907
2024-09-11 20:52:35.778701: W external/local_xla/xla/service/gpu/nvptx_compiler.cc:762] The NVIDIA driver's CUDA version is 12.2 which is older than the ptxas CUDA version (12.3.107). Because the driver is older than the ptxas version, XLA is disabling parallel compilation, which may slow down compilation. You should update your NVIDIA driver or use the NVIDIA-provided CUDA forward compatibility

 22/420 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step

I0000 00:00:1726098756.303863    5055 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


420/420 ━━━━━━━━━━━━━━━━━━━━ 14s 17ms/step


In [5]:
predicted_labels = np.argmax(results, axis=1)

In [6]:
_, predicted_ammount = np.unique(predicted_labels, return_counts=True)

In [7]:
precision, recall, fscore, support = precision_recall_fscore_support(true_labels, predicted_labels, average=None)

In [8]:
auc = roc_auc_score(true_labels, results, average=None, multi_class='ovr')

In [9]:
acc = accuracy_score(true_labels, predicted_labels) * 100

print(f"Test dataset accuracy: {acc:.2f}%")

Test dataset accuracy: 99.28%


In [10]:
import pandas as pd

pd.DataFrame(
    {"class": class_names, "support": support, "predicted_ammount": predicted_ammount, "precision": precision, "recall": recall, "fscore": fscore, "auc": auc}
)

,class,support,predicted_ammount,precision,recall,fscore,auc
0,confirmed,4463,4434,0.994587,0.988125,0.991345,0.999549
1,crossedout,80,107,0.495327,0.662500,0.566845,0.984531
2,empty,8876,8878,0.997860,0.998085,0.997972,0.999937


In [11]:
cm = confusion_matrix(true_labels, predicted_labels)

df = pd.DataFrame(cm, index=class_names, columns=class_names)
df.index.name = "True"
df.columns.name = "Predicted"
df

Predicted,confirmed,crossedout,empty
True,,,
confirmed,4410,43,10
crossedout,18,53,9
empty,6,11,8859
